### This work is the EDA part for incidents analytics work
- The dataset are downloaded from SSR report. This one is the incidents dataset

In [2]:
# import required labraries
import pandas as pd 
import numpy as np

# Read in incident data from excel sheet. This data set is downloaded from SSR report
df = pd.read_csv("/opt/data/share01/jl2408/2018incidents.csv")
# list column names
cln = list(df)
print(cln)
s_cln = ['INCIDENT_TICKET_NUMBER', 'ROOT_CAUSE', 'SUB_CAUSE', 'PBLM_CAUSED_BY_CHANGE', 'OUTAGE_CAUSER', 'START_DATE', 'START_TIME', 'END_DATE', 'END_TIME', 'INCIDENT_DURATION', 'ALERTED', 'SEVERITY_CLASS', 'DOWNTIME_TYPE', 'SERVICE_COMPONENT', 'AVP_ATTUID', 'AVP_NAME', 'VP_ATTUID', 'VP_NAME', 'SVP_ATTUID', 'SVP_NAME', 'COE_NAME', 'COE_ORG', 'RCO_SVP_VP', 'IMPACTED_APPS']
id_cln = s_cln[:]
id_cln.remove('IMPACTED_APPS')
s_cln2 = ['INCIDENT_TICKET_NUMBER', 'ROOT_CAUSE', 'SUB_CAUSE', 'PBLM_CAUSED_BY_CHANGE', 'OUTAGE_CAUSER', 'START_DATE', 'START_TIME', 'END_DATE', 'END_TIME', 'INCIDENT_DURATION', 'ALERTED', 'SEVERITY_CLASS', 'DOWNTIME_TYPE', 'SERVICE_COMPONENT', 'IMPACTED_APPS']
print(id_cln)
# Example data in train
df.head()

In [3]:
# Data information in train
df.info()

In [4]:
# Get a subset of the data
df = df[s_cln]

In [5]:
# Number of unique values
df.apply(pd.Series.nunique)

In [6]:
# Check for missing values
print(df.isnull().sum())

In [7]:
# data set info
df.info()

In [8]:
# remove duplicated rows
df1 = df.drop_duplicates()
df1.info()

In [9]:
# check unique values in ROOT_CAUSE
df1.ROOT_CAUSE.unique()

In [10]:
# check unique values in SUB_CAUSE
df1.SUB_CAUSE.unique()

In [11]:
# Inconsistent content in IMPACTED_APPS
# For example, we see multiple impacted applications in ticket 253909391-1

df1[["INCIDENT_TICKET_NUMBER","IMPACTED_APPS"]]

In [12]:
# Check multi-impacted-apps from IMPACTED_APPS
import re
m_apps = df1["IMPACTED_APPS"].apply(lambda x: re.findall("[^,]+,+[^,]+,+[^,]+",str(x)))
m_apps[m_apps.str.len() != 0].head(10)

In [13]:
# We need to split these into multiple rows in order to get correct and consistent impacted application metrics
df2= df1["IMPACTED_APPS"].str.split(",",expand = True) \
.merge(df1, right_index = True, left_index = True) \
.drop(["IMPACTED_APPS"], axis = 1)
df2 = pd.melt(df2, id_vars = id_cln, value_name = "IMPACTED_APPS")
df2 = df2.drop("variable", axis = 1)

In [14]:
pd.__version__

In [15]:
# data set info
df2.info()

In [16]:
# a log of none value created in IMPACTED_APPS
df2.isnull().sum()

In [17]:
# drop rows associated with these None IMPACTED_APPS
import numpy as np
df2 = df2[pd.notnull(df2['IMPACTED_APPS'])]

In [18]:
# a log of none value created in IMPACTED_APPS
df2.isnull().sum()

In [19]:
# drop duplicate rows
df2.drop_duplicates(inplace=True)

In [20]:
# a log of none value created in IMPACTED_APPS
df2.isnull().sum()

In [21]:
df2.info()

In [22]:
df2.to_excel("/opt/data/share01/jl2408/tmp.xlsx")

In [23]:
df3 = pd.read_excel("/opt/data/share01/jl2408/tmp.xlsx")

In [24]:
df3.drop_duplicates(inplace=True)

In [25]:
df3.info()